In [1]:
# The standard library modules
import os
import sys

# The wget module
import wget

# The BeautifulSoup module
from bs4 import BeautifulSoup
import re

# The selenium module
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import stem
from stem import Signal
from stem.control import Controller
from stem.connection import connect
import time

executable_path='/home/bhorkar/node_modules/phantomjs-prebuilt/lib/phantom/bin//phantomjs'


In [2]:
'''
Python script to connect to Tor via Stem and Privoxy, requesting a new connection (hence a new IP as well) as desired.
'''

import stem
import stem.connection

import time
import urllib2

from stem import Signal
from stem.control import Controller

# initialize some HTTP headers
# for later usage in URL requests
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent}


# initialize some
# holding variables
oldIP = "0.0.0.0"
newIP = "0.0.0.0"

# how many IP addresses
# through which to iterate?
nbrOfIpAddresses = 1

# seconds between
# IP address checks
secondsBetweenChecks = 2

# request a URL 
def request(url):
    # communicate with TOR via a local proxy (privoxy)
    def _set_urlproxy():
        proxy_support = urllib2.ProxyHandler({"http" : "127.0.0.1:8118"})
        opener = urllib2.build_opener(proxy_support)
        urllib2.install_opener(opener)

    # request a URL
    # via the proxy
    _set_urlproxy()
    request=urllib2.Request(url, None, headers)
    return urllib2.urlopen(request).read()

# signal TOR for a new connection 
def renew_connection():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password = 'garbage123')
        controller.signal(Signal.NEWNYM)
        controller.close()
        


ua_list = []
with open('ualist.txt') as ua:
    for line in ua:
        ua_list.append(line.strip())
#print ua_list
ua_list_len = len(ua_list)

import random

#def request(url,ua_list = ua_list,ua_list_len = ua_list_len):
#    ua = ua_list[random.randint(0,ua_list_len-1)]
#    headers={'User-Agent': ua}
#    request=urllib2.Request(url, None, headers)
#    return urllib2.urlopen(request).read()        


In [3]:
from TorCtl import TorCtl
import urllib2
 
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent}
 
def request(url):
    def _set_urlproxy():
        proxy_support = urllib2.ProxyHandler({"http" : "127.0.0.1:8118"})
        opener = urllib2.build_opener(proxy_support)
        urllib2.install_opener(opener)
    _set_urlproxy()
    request=urllib2.Request(url, None, headers)
    return urllib2.urlopen(request).read()
 
def renew_connection():
    conn = TorCtl.connect(controlAddr="127.0.0.1", controlPort=9051, passphrase="garbage123")
    conn.send_signal("NEWNYM")
    conn.close()
 
for i in range(0, 10):
    renew_connection()
    print request("http://icanhazip.com/")

INFO[Thu Jun 01 23:40:16 2017]:Event loop received close message.
146.185.177.103

INFO[Thu Jun 01 23:40:28 2017]:Event loop received close message.
62.102.148.67

INFO[Thu Jun 01 23:40:29 2017]:Event loop received close message.
62.102.148.67

INFO[Thu Jun 01 23:40:31 2017]:Event loop received close message.
62.102.148.67

INFO[Thu Jun 01 23:40:32 2017]:Event loop received close message.
62.102.148.67

INFO[Thu Jun 01 23:40:35 2017]:Event loop received close message.
62.102.148.67

INFO[Thu Jun 01 23:40:36 2017]:Event loop received close message.
62.102.148.67

INFO[Thu Jun 01 23:40:38 2017]:Event loop received close message.
163.172.223.200

INFO[Thu Jun 01 23:40:39 2017]:Event loop received close message.
163.172.223.200

INFO[Thu Jun 01 23:40:40 2017]:Event loop received close message.
INFO[Thu Jun 01 23:40:40 2017]:Closed control connection. Exiting thread.
163.172.223.200



In [ ]:

# cycle through
# the specified number
# of IP addresses via TOR 
def changeip(newIP):
 for i in range(0, nbrOfIpAddresses):

    # if it's the first pass
    if newIP == "0.0.0.0":
        # renew the TOR connection
        renew_connection()
        # obtain the "new" IP address
        newIP = request("http://icanhazip.com/")
    # otherwise
    else:
        # remember the
        # "new" IP address
        # as the "old" IP address
        oldIP = newIP
        # refresh the TOR connection
        renew_connection()
        # obtain the "new" IP address
        newIP = request("http://icanhazip.com/")
        print newIP

    # zero the 
    # elapsed seconds    
    seconds = 0

    # loop until the "new" IP address
    # is different than the "old" IP address,
    # as it may take the TOR network some
    # time to effect a different IP address
    while oldIP == newIP:
        # sleep this thread
        # for the specified duration
        time.sleep(secondsBetweenChecks)
        # track the elapsed seconds
        seconds += secondsBetweenChecks
        # obtain the current IP address
        newIP = request("http://icanhazip.com/")
        # signal that the program is still awaiting a different IP address
        #print ("%d seconds elapsed awaiting a different IP address." % seconds)
    # output the
    # new IP address
 #   print ("")
    print ("newIP: %s" % newIP)
    return 

In [ ]:
newIP = changeip(newIP);

In [33]:
import requests

In [34]:
def loading_link(items):
 shop_urls = [];
 for elem in items:
   # print elem.get_attribute("id")
    id = elem.get_attribute("id");
    img_count = 0;
    try:
     street_links = elem.find_elements_by_xpath(".//img[contains(@class, 'ltk-image')]")
     idx = 0;
     for sl in street_links:
           idx = idx + 1;
           #print sl.get_attribute("src")
           response = requests.get(sl.get_attribute('src')).content
           with open('street/%s.jpeg' % ( id + '_' + str(idx)), 'wb') as f:
                         f.write(response)
     shop_links = elem.find_elements_by_xpath(".//a[contains(@href, ')]")
     url_idx = 0;
     for sl in shop_links:
        url_idx = url_idx + 1;
        shop_urls.append((id,url_idx,sl.get_attribute("href")))
    except:
      continue;   
  #  break;
 #print "loading #urls", len(shop_urls)
 #print shop_urls;
 return shop_urls;


In [6]:
from selenium import webdriver
link = "https://www.liketoknow.it/stylethegirl" # load the web page

driver = webdriver.PhantomJS(executable_path = executable_path)
# it takes forever to load the page, therefore we are setting a threshold
driver.set_page_load_timeout(500)
driver.get(link)




In [25]:
driver.get(link)

In [26]:
# Load Twitter page and click to view all results

#lems = driver.find_elements_by_xpath("//a[contains(@href,'rstyle.me')]")
 
street_urls = [];
pause = 1;
lastHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(pause)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight


In [27]:

items = driver.find_elements_by_xpath("//div[contains(@class, 'ltk-item')]")



In [28]:
print "Number of items {}".format(len(items))


Number of items 840


In [35]:

shop_urls = loading_link(items);

In [36]:
print "number of shop urls {}".format(len(shop_urls))


number of shop urls 7148


In [ ]:
from time import sleep 
from multiprocessing import Pool
import sys

directory = './shop/'

if not os.path.exists(directory):
    os.makedirs(directory)

def job(url):
  try:
   # print url
    driver = webdriver.PhantomJS(executable_path = executable_path)
    driver.set_page_load_timeout(100)
    
    
    driver.get(url[2]);
    slp_flt =   random.uniform(10,15)                                                                                 
    #print 'sleeping',slp_flt                                                                                        
    sleep(slp_flt)                                                                                                                                                                                                                                          
    pgsrc  = driver.page_source 
    soup = BeautifulSoup(pgsrc, 'html.parser')
    fname= directory + str(url[0]) + '_'  + str(url[1]) + "_image-gallery.html" 
    outf = open(fname,"w")                                                                                          
    outf.write(pgsrc.encode('utf-8'))
    driver.close();
  except:
    print "cannot complete", url 
    print "Unexpected error:", sys.exc_info()[0]

 

In [ ]:
count = 0;
#newIP = changeip(newIP);
renew_connection()
for url in shop_urls:
   count = count + 1
   job(url)
   if (count%20 == 0):
    try: 
      renew_connection();
    except:
        pass;

INFO[Thu Jun 01 23:58:38 2017]:Event loop received close message.


In [ ]:
#print shop_urls[0:3]
#pool.map(job, shop_urls)

In [ ]:
import glob
import wget

from BeautifulSoup import BeautifulSoup as bs
import urlparse
from urllib2 import urlopen
from urllib import urlretrieve
import os
import sys
import socket

socket.setdefaulttimeout(30)

def main(url, out_folder="./street/", local = 1):
    soup = bs(open(url))
    #print url
    count = 0;
    for image in soup.findAll("img"):
        #print image
        count = count + 1;
        try: 
          #print image["src"].split(".")[-1]
         
          filename = url.split('/')[2].split('.')[0] +  + '_' + str(count) + '_'+'.'+image["src"].split(".")[-1]
            
        except:
          #  print "image not found", 
            continue
        image_url = urlparse.urljoin(url, image['src'])
        try: 
           #print out_folder+filename
           urlretrieve(image_url,out_folder+filename)
        except:
            pass
 

          

            

In [ ]:


out_folder = "./shop/"
urls = glob.glob(out_folder+'/*.html');
for i in xrange(1, len(urls)):
    main(urls[i], out_folder)




In [ ]:
#from multiprocessing import Pool

#pool = Pool()
#pool.map(main, urls)

In [ ]:
#for img in soup2.findAll('a href'):
#    print img

In [ ]:
#import re
#urls = re.findall(r'href=[\'"]?(http://rstyle.me[^\'" >]+)', soup2.prettify())
#print ', '.join(urls)
